In [1]:
# 코드 7-24 모델을 생성하고 훈련
import numpy as np
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import imdb

2024-12-03 01:48:10.590155: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-03 01:48:10.592875: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-03 01:48:10.600881: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733158090.615454  107845 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733158090.619657  107845 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-03 01:48:10.635986: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [2]:
n_unique_words = 10000
maxlen = 200
batch_size = 128

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=n_unique_words)
# 데이터 길이가 같지 않을 때 일정한 길이로 맞추어 줍니다.
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
# y_train을 배열로 생성
y_train = np.array(y_train)
y_test = np.array(y_test)

model = Sequential()
model.add(Embedding(n_unique_words, 128, input_length=maxlen))
model.add(Bidirectional(LSTM(64)))  # 양방향 LSTM
model.add(Dropout(0.5))  # 드롭아웃 50%만 모델에 전달
model.add(Dense(1, activation="sigmoid"))  # 이진 분류

model.compile(loss="binary_crossentropy", optimizer=Adam(), metrics=["accuracy"])

model.fit(
    x_train, y_train, batch_size=batch_size, epochs=4, validation_data=[x_test, y_test]
)

Epoch 1/4


/home/halozhan/Study/.venv/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
W0000 00:00:1733158096.149482  107845 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


196/196 ━━━━━━━━━━━━━━━━━━━━ 40s 189ms/step - accuracy: 0.6908 - loss: 0.5544 - val_accuracy: 0.8583 - val_loss: 0.3420
Epoch 2/4
196/196 ━━━━━━━━━━━━━━━━━━━━ 37s 187ms/step - accuracy: 0.9022 - loss: 0.2534 - val_accuracy: 0.8734 - val_loss: 0.2987
Epoch 3/4
196/196 ━━━━━━━━━━━━━━━━━━━━ 33s 169ms/step - accuracy: 0.9347 - loss: 0.1787 - val_accuracy: 0.8647 - val_loss: 0.3318
Epoch 4/4
196/196 ━━━━━━━━━━━━━━━━━━━━ 33s 166ms/step - accuracy: 0.9463 - loss: 0.1481 - val_accuracy: 0.8464 - val_loss: 0.3741


In [3]:
# 코드 7-25 LSTM 모델 구조 확인
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 200, 128)       │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 128)            │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,136,837 (15.78 MB)

 Trainable params: 1,378,945 (5.26 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,757,892 (10.52 MB)

In [4]:
# 코드 7-26 모델 평가
loss, acc = model.evaluate(x_test, y_train, batch_size=384, verbose=1)
print("Training accuracy", model.metrics_names, acc)
print("Training loss", model.metrics_names, loss)
loss, acc = model.evaluate(x_test, y_test, batch_size=384, verbose=1)
print("Testing accuracy", model.metrics_names, acc)
print("Testing loss", model.metrics_names, loss)

66/66 ━━━━━━━━━━━━━━━━━━━━ 7s 106ms/step - accuracy: 0.4949 - loss: 1.7297
Training accuracy ['loss', 'compile_metrics'] 0.4986000061035156
Training loss ['loss', 'compile_metrics'] 1.7240833044052124
66/66 ━━━━━━━━━━━━━━━━━━━━ 7s 109ms/step - accuracy: 0.8451 - loss: 0.3825
Testing accuracy ['loss', 'compile_metrics'] 0.8463600277900696
Testing loss ['loss', 'compile_metrics'] 0.374111533164978
